In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd
import pickle


In [ ]:
data_path = None

In [ ]:
#data_path = '/home/myusername/data' # On Linux, if you now have the folder /home/myusername/data/Bungendore
#data_path = r'C:\data\Lithology'  # windows, if you have C:\data\Lithology\Bungendore

In [ ]:
if data_path is None:
    if ('ELA_DATA' in os.environ):
        data_path = os.environ['ELA_DATA']
    elif sys.platform == 'win32':
        data_path = r'C:\data\Lithology'
    else:
        username = os.environ['USER']
        data_path = os.path.join('/home', username, 'data')

In [ ]:
example_folder = "./examples"
examples = [f for f in os.listdir(example_folder) if f.endswith('.gpx')]

In [ ]:
data_path

In [ ]:
cbr_datadir = os.path.join(data_path, 'Canberra')
cbr_datadir_out = os.path.join(cbr_datadir, 'out')
ngis_datadir = os.path.join(data_path, 'NGIS')
act_shp_datadir = os.path.join(ngis_datadir, 'shp_ACT')
bidgee_shp_datadir = os.path.join(ngis_datadir, 'shp_murrumbidgee_river')

In [ ]:
# classified_logs_filename = os.path.join(cbr_datadir_out,'classified_logs.pkl')
# with open(classified_logs_filename, 'rb') as handle:
#     df = pickle.load(handle)

In [ ]:
df = pd.read_csv(os.path.join(cbr_datadir_out,'classified_logs.csv'))

In [ ]:
# geoloc_filename = os.path.join(cbr_datadir_out,'geoloc.pkl')
# with open(geoloc_filename, 'rb') as handle:
#     geoloc = pickle.load(handle)

In [ ]:
geoloc = pd.read_csv(os.path.join(cbr_datadir_out,'geoloc.csv'))

In [ ]:
DEPTH_FROM_COL = 'FromDepth'
DEPTH_TO_COL = 'ToDepth'

TOP_ELEV_COL = 'TopElev'
BOTTOM_ELEV_COL = 'BottomElev'

LITHO_DESC_COL = 'Description'
HYDRO_CODE_COL = 'HydroCode'

HYDRO_ID_COL = 'HydroID'
BORE_ID_COL = 'BoreID'

In [ ]:
# if we want to keep vboreholes that have more than one row
x = df[HYDRO_ID_COL].values
unique, counts = np.unique(x, return_counts=True)
multiple_counts = unique[counts > 1]
len(multiple_counts), len(unique)

In [ ]:
keep = set(df[HYDRO_ID_COL].values)
keep = set(multiple_counts)
s = geoloc[HYDRO_ID_COL]

In [ ]:
geoloc = geoloc[s.isin(keep)]

# Viewer

Derived from [voila-gpx-viewer](https://github.com/jtpio/voila-gpx-viewer) and [ipyleaflet docs](https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html)

However this is very difficult to adapt, actually use case is quite different from the GPX data. ipyleaflet doc is sparse and how to I guess the content of callbacks??
https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6




In [ ]:
# from bqplot import Axis, Figure, Lines, LinearScale
# from bqplot.interacts import IndexSelector
# from ipyleaflet import basemaps, FullScreenControl, LayerGroup, Map, MeasureControl, Polyline, Marker, MarkerCluster, CircleMarker, WidgetControl
# from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link
from ipywidgets import Output, HTML
from ipyleaflet import Map, Marker, MarkerCluster, basemaps

In [ ]:
import ipywidgets as widgets
from ipysheet import sheet, cell, column
import ipysheet

In [ ]:
class GlobalThing:
    def __init__(self, bore_data, displayed_colnames = None):
        self.marker_info = dict()
        self.bore_data = bore_data
        if displayed_colnames is None:
            displayed_colnames = [BORE_ID_COL, DEPTH_FROM_COL, DEPTH_TO_COL, LITHO_DESC_COL] # 'Lithology_1', 'MajorLithCode']]
        self.displayed_colnames = displayed_colnames
    
    def add_marker_info(self, lat, lon, code):
        self.marker_info[(lat, lon)] = code
    
    def get_code(self, lat, lon):
        return self.marker_info[(lat, lon)]

    def data_for_hydroid(self, ident):
        df_sub = self.bore_data.loc[df[HYDRO_ID_COL] == ident]
        return df_sub[self.displayed_colnames]

    def register_geolocations(self, geoloc):
        for index, row in geoloc.iterrows():
            self.add_marker_info(row.Latitude, row.Longitude, row.HydroID)

In [ ]:
globalthing = GlobalThing(df, displayed_colnames = [BORE_ID_COL, DEPTH_FROM_COL, DEPTH_TO_COL, LITHO_DESC_COL, 'Lithology_1'])
globalthing.register_geolocations(geoloc)

In [ ]:
def plot_map(geoloc, click_handler):
    """
    Plot the markers for each borehole, and register a custom click_handler
    """
    mean_lat = geoloc.Latitude.mean()
    mean_lng = geoloc.Longitude.mean()
    # create the map
    m = Map(center=(mean_lat, mean_lng), zoom=12, basemap=basemaps.Stamen.Terrain)
    m.layout.height = '600px'
    # show trace
    markers = []
    for index, row in geoloc.iterrows():
        message = HTML()
        message.value = str(row.HydroID)
        message.placeholder = ""
        message.description = "HydroID"
        marker = Marker(location=(row.Latitude, row.Longitude))
        marker.on_click(click_handler)
        marker.popup = message
        markers.append(marker)
    marker_cluster = MarkerCluster(
        markers=markers
    )
    # not sure whether we could register once instead of each marker:
    # marker_cluster.on_click(click_handler)
    m.add_layer(marker_cluster);
    # m.add_control(FullScreenControl())
    return m

In [ ]:
# If printing a data frame straight to an output widget
def raw_print(out, ident):
    bore_data = globalthing.data_for_hydroid(ident)
    out.clear_output()
    with out:
        print(ident)        
        print(bore_data)
        
def click_handler_rawprint(**kwargs):
    blah = dict(**kwargs)
    xy = blah['coordinates']
    ident = globalthing.get_code(xy[0], xy[1])
    raw_print(out, ident)


In [ ]:
# to display using an ipysheet
def mk_sheet(d):
    return ipysheet.pandas_loader.from_dataframe(d)

def upate_display_df(ident):
    bore_data = globalthing.data_for_hydroid(ident)
    out.clear_output()
    with out:
        display(mk_sheet(bore_data))        

def click_handler_ipysheet(**kwargs):
    blah = dict(**kwargs)
    xy = blah['coordinates']
    ident = globalthing.get_code(xy[0], xy[1])
    upate_display_df(ident)


In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})

In [ ]:
plot_map(geoloc, click_handler_ipysheet)
# plot_map(geoloc, click_handler_rawprint)

In [ ]:
out

In [ ]:
## Appendix A : qgrid, but at best ended up with "Model not available". May not work yet with Jupyter lab 1.0.x

# import qgrid

# d = data_for_hydroid(10062775)
# d

# import ipywidgets as widgets

# def build_qgrid():
#     qgrid.set_grid_option('maxVisibleRows', 10)
#     col_opts = { 
#         'editable': False,
#     }
#     qgrid_widget = qgrid.show_grid(d, show_toolbar=False, column_options=col_opts)
#     qgrid_widget.layout = widgets.Layout(width='920px')
#     return qgrid_widget, qgrid

# qgrid_widget, qgrid = build_qgrid()

# display(qgrid_widget)

# pitch_app = widgets.VBox(qgrid_widget)
# display(pitch_app)

# def click_handler(**kwargs):
#     blah = dict(**kwargs)
#     xy = blah['coordinates']
#     ident = globalthing.get_code(xy[0], xy[1])
#     bore_data = data_for_hydroid(ident)
#     grid.df = bore_data

In [ ]:

## Appendix B: using striplog 

# from striplog import Striplog, Interval, Component, Legend, Decor


# import matplotlib as mpl
# lithologies = ['shale', 'clay','granite','soil','sand', 'porphyry','siltstone','gravel', '']
# lithology_color_names = ['lightslategrey', 'olive', 'dimgray', 'chocolate',  'gold', 'tomato', 'teal', 'lavender', 'black']
# lithology_colors = [mpl.colors.cnames[clr] for clr in lithology_color_names]

# clrs = dict(zip(lithologies, lithology_colors))

# def mk_decor(lithology, component):
#     dcor = {'color': clrs[lithology],
#         'component': component,
#         'width': 2}
#     return Decor(dcor)

# def create_striplog_itvs(d):
#     itvs = []
#     dcrs = []
#     for index, row in d.iterrows():
#         litho = row.Lithology_1
#         c = Component({'description':row.Description,'lithology': litho})
#         decor = mk_decor(litho, c)
#         itvs.append(Interval(row.FromDepth, row.ToDepth, components=[c]) )
#         dcrs.append(decor)
#     return itvs, dcrs

# def click_handler(**kwargs):
#     blah = dict(**kwargs)
#     xy = blah['coordinates']
#     ident = globalthing.get_code(xy[0], xy[1])
#     bore_data = data_for_hydroid(ident)
#     itvs, dcrs = create_striplog_itvs(bore_data)
#     s = Striplog(itvs)
#     with out:
#         print(ident)
#         print(s.plot(legend = Legend(dcrs)))

# def plot_striplog(bore_data, ax=None):
#     itvs, dcrs = create_striplog_itvs(bore_data)
#     s = Striplog(itvs)
#     s.plot(legend = Legend(dcrs), ax=ax)

# def plot_evaluation_metrics(bore_data):
#     fig, ax = plt.subplots(figsize=(12, 3))
#     # actual plotting
#     plot_striplog(bore_data, ax=ax)
#     # finalize
#     fig.suptitle("Evaluation metrics with cutoff\n", va='bottom')
#     plt.show()
#     plt.close(fig)

# %matplotlib inline
# from ipywidgets import interactive
# import matplotlib.pyplot as plt
# import numpy as np

# def f(m, b):
#     plt.figure(2)
#     x = np.linspace(-10, 10, num=1000)
#     plt.plot(x, m * x + b)
#     plt.ylim(-5, 5)
#     plt.show()

# interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
# output = interactive_plot.children[-1]
# output.layout.height = '350px'
# interactive_plot

# def update_sheet(s, d):
#     print("before: %s"%(s.rows))
#     s.rows = len(d)
#     for i in range(len(d.columns)):
#         s.cells[i].value = d[d.columns[i]].values